In [1]:
import os
import pandas as pd
import numpy as np
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch
import torch.nn as nn
import torch.optim as optim
import random
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torch.cuda.amp import autocast, GradScaler
import cv2

# 하이퍼파라미터 설정
num_epochs = 50
batch_size = 16
learning_rate = 0.001

# 시드값 설정
def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

set_seed(42)

# 데이터셋 클래스 정의
class ImageDataset(Dataset):
    def __init__(self, image_dir, csv_file=None, transform=None):
        self.image_dir = image_dir
        self.csv_file = csv_file
        self.transform = transform
        if csv_file is not None:
            self.df = pd.read_csv(csv_file)
        else:
            self.df = None
        self.image_names = os.listdir(image_dir)
    
    def __len__(self):
        return len(self.image_names)
    
    def __getitem__(self, idx):
        img_name = self.image_names[idx]
        img_path = os.path.join(self.image_dir, img_name)
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        if self.df is not None:
            label = self.df[self.df['ID'] == img_name]['target'].values[0]
            return image, label
        else:
            return image

# 이미지 전처리 정의
transform = transforms.Compose([
    transforms.Resize((512, 512)),
    transforms.ToTensor()
])
        

# 오토인코더 모델 정의
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(64, 32, kernel_size=2, stride=2),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 32, kernel_size=2, stride=2),
            nn.ReLU(),
            nn.Conv2d(32, 3, kernel_size=3, stride=1, padding=1),
            nn.Sigmoid()
        )


    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

# GPU가 사용 가능한지 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

autoencoder = Autoencoder().to(device)    

# 테스트 데이터셋 로드
class TestDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.file_names = [f for f in os.listdir(root_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
        self.transform = transform

    def __len__(self):
        return len(self.file_names)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.file_names[idx])
        image = Image.open(img_name).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, self.file_names[idx]

test_dataset = TestDataset(root_dir='cv_challenge/test', transform=transform)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# 결과를 저장할 폴더 생성
output_dir = 'origin_denoising_AE'
os.makedirs(output_dir, exist_ok=True)

# 모델 로드
autoencoder.load_state_dict(torch.load('autoencoder_model_512.pth'))
autoencoder.eval()

# 테스트 데이터 노이즈 제거 및 저장
with torch.no_grad():
    for images, file_names in test_loader:
        images = images.to(device)
        outputs = autoencoder(images)
        outputs = outputs.cpu().numpy()
        for i, file_name in enumerate(file_names):
            denoised_image = outputs[i].transpose(1, 2, 0)  # [C, H, W]에서 [H, W, C]로 변환
            denoised_image = (denoised_image * 255).astype(np.uint8)  # [0, 1] 범위를 [0, 255]로 변환
            denoised_image = np.clip(denoised_image, 0, 255)  # 값이 [0, 255] 범위 내에 있는지 확인
            output_path = os.path.join(output_dir, file_name)
            Image.fromarray(denoised_image).save(output_path)

C:\Users\master\anaconda3\envs\EDA\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO:albumentations.check_version:A new version of Albumentations is available: 1.4.12 (you have 1.4.10). Upgrade using: pip install --upgrade albumentations


Using device: cuda
